# Producing weights for likelihood grid calculation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
import pickle
import gc

from exp_analysis_class import full_likelihood
from dark_nus_utils import load_datasets
from const import alphaQED
from analyses_dict import analyses
from parameters_dict import *
from dark_nus_utils import store_events_weights, retrieve_events_weights

In [3]:
D_or_M = 'majorana'

# save_folder = f'./likelihood_weights/'
# !mkdir -p $save_folder

In [4]:
# my_exp_analyses = load_datasets(hierarchies=['heavy', 'light'],
#                                   D_or_Ms=D_or_M,
#                                   fluxes=['FHC', 'RHC'],
#                                 timeit=True, 
#                                 direct_load_objects=False,
#                                 load_grid=False,
#                                 dump=True,
#                                 nentries=1000000)

In [5]:
my_exp_analyses = load_datasets(
                                # hierarchies=['heavy', 'light'], 
                                hierarchies='light', 
                                D_or_Ms=D_or_M, 
                                fluxes=['FHC', 'RHC'],
                                timeit=True, 
                                direct_load_objects=True,
                                load_grid=False,
                                dump=False,
                                nentries=1000000)

light majorana FHC
Wall time: 4.550288677215576 s, CPU time: 0.4466171349999999
light majorana RHC
Wall time: 4.567636251449585 s, CPU time: 0.44681856799999986


## Produce event weights

In [6]:
for case_name, case in likelihood_calculation_pars.items():
    if 'heavy' in case_name:
        continue
    name = case['name']
    hierarchy = case['hierarchy']
    analysis_name = case['analysis_name']
    pars = case['pars']
    additional_vars = case['additional_vars']
    for nu_mode in ['FHC', 'RHC']:
    # for nu_mode in ['RHC']:
        print(name, analysis_name, nu_mode)
        out = my_exp_analyses[f'{hierarchy}_{D_or_M}_{nu_mode}'].compute_expectation(
                                                            my_exp_analyses[f'{hierarchy}_{D_or_M}_{nu_mode}'].df_base, 
                                                            **pars,
                                                            ntarget_per_material=None, 
                                                            pot=None,
                                                            selection_query=analyses[analysis_name][nu_mode]['selection'],
                                                            efficiency_factor=None,
                                                            no_pot_efficiency=True)
        store_events_weights(expectation_output=out, 
                             analysis=analyses[analysis_name][nu_mode],
                             pars=pars,
                             name=f'{hierarchy}_{name}',
                             analysis_name=analysis_name,
                             nu_mode=nu_mode,
                             additional_vars=additional_vars)
        del out
        gc.collect()

mz_epsilon nueccqe_fgd FHC
mz_epsilon nueccqe_fgd RHC
m4_Umu4_2 nueccqe_fgd FHC
m4_Umu4_2 nueccqe_fgd RHC
